In [1]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/mustafaonurertugrul/Documents/Projects/insurance_project/vertex_key.json"
)

from dotenv import load_dotenv
load_dotenv()

print("GOOGLE_APPLICATION_CREDENTIALS =", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))



GOOGLE_APPLICATION_CREDENTIALS = /Users/mustafaonurertugrul/Documents/Projects/insurance_project/vertex_key.json


In [2]:
import vertexai
import google.cloud.aiplatform as aiplatform

print(vertexai.__version__)
print(aiplatform.__version__)


1.71.1
1.71.1


In [3]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/mustafaonurertugrul/Documents/Projects/insurance_project/vertex_key.json"
)

os.environ["VERTEX_PROJECT_ID"] = "insurance-vertex"
os.environ["VERTEX_LOCATION"] = "us-central1"



In [4]:
from google.auth import default
creds, project = default()
print("Authenticated as:", creds.service_account_email)
print("Project:", project)


Authenticated as: vertex-runner@insurance-vertex.iam.gserviceaccount.com
Project: insurance-vertex


In [7]:
from google import genai
from google.genai import types

# Use 'http_options' to ensure no proxy or versioning interference
client = genai.Client(
    api_key="AIza...", 
    http_options={'api_version': 'v1beta'} # Sometimes 'v1' is stricter
)

try:
    response = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents="Hello"
    )
    print("✅ SUCCESS!")
except Exception as e:
    print(f"❌ FAILED: {e}")
    

❌ FAILED: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}


In [2]:
from rag.retriever_xgboost import InsuranceRAG_XGB
from agents.frontier_agent import FrontierAgent
from agents.gemini_frontier_agent import GeminiFrontierAgent
from agents.ensemble_agent_light import EnsemblePredictor

from dotenv import load_dotenv
load_dotenv()

# 1. RAG retriever
rag = InsuranceRAG_XGB(
    model_path="./rag/xgb_model.json",
    db_path="./chroma_db",
    collection_name="insurance_cases_xgb",
)

# 2. FrontierAgent (OpenAI)
frontier = FrontierAgent(
    model="gpt-4o-mini",
    rag=rag,
)

# 3. GeminiFrontierAgent (Gemini)
gemini = GeminiFrontierAgent(
    model="gemini-pro",
    rag=rag,
)

# 4. EnsemblePredictor (XGBoost + Frontier + Gemini)
ensemble = EnsemblePredictor(
    xgb_model_path="./rag/xgb_model.json",
    frontier_agent=frontier,
    gemini_agent=gemini,   # <-- REQUIRED NOW
)

# 5. Sample input
features = {
    "age": 40,
    "sex": "male",
    "bmi": 28.5,
    "children": 2,
    "smoker": "no",
}

# 6. Predict
result = ensemble.predict(features)
print(result)


NotFound: 404 Publisher Model `projects/insurance-vertex/locations/us-central1/publishers/google/models/gemini-pro` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions